# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and implementation of two (simple) techniques that are enough to solve sudoku in a reasonable time.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [1]:
#notation

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

print(units['A1'])

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]


### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:



In [2]:
# parser
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print()

# sample sudoku
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
display(grid_values(grid2))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 



### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [3]:
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (grid_values(grid2))
                 
print(sudoku_values)

{'A1': '4', 'A2': '.', 'A3': '.', 'A4': '.', 'A5': '.', 'A6': '.', 'A7': '8', 'A8': '.', 'A9': '5', 'B1': '.', 'B2': '3', 'B3': '.', 'B4': '.', 'B5': '.', 'B6': '.', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '.', 'C3': '.', 'C4': '7', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '.', 'C9': '.', 'D1': '.', 'D2': '2', 'D3': '.', 'D4': '.', 'D5': '.', 'D6': '.', 'D7': '.', 'D8': '6', 'D9': '.', 'E1': '.', 'E2': '.', 'E3': '.', 'E4': '.', 'E5': '8', 'E6': '.', 'E7': '4', 'E8': '.', 'E9': '.', 'F1': '.', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '1', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '.', 'G1': '.', 'G2': '.', 'G3': '.', 'G4': '6', 'G5': '.', 'G6': '3', 'G7': '.', 'G8': '7', 'G9': '.', 'H1': '5', 'H2': '.', 'H3': '.', 'H4': '2', 'H5': '.', 'H6': '.', 'H7': '.', 'H8': '.', 'H9': '.', 'I1': '1', 'I2': '.', 'I3': '4', 'I4': '.', 'I5': '.', 'I6': '.', 'I7': '.', 'I8': '.', 'I9': '.'}


# Constraint Propagation - original code
The function parse_grid calls assign(values, s, d). We could implement this as values[s] = d, but we can do more than just that. Those with experience solving Sudoku puzzles know that there are two important strategies that we can use to make progress towards filling in all the squares:

(1) If a square has only one possible value, then eliminate that value from the square's peers. 

(2) If a unit has only one possible place for a value, then put the value there.

As an example of strategy (1) if we assign 7 to A1, yielding {'A1': '7', 'A2':'123456789', ...}, we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers), giving us {'A1': '7', 'A2': '12345689', ...}. As an example of strategy (2), if it turns out that none of A3 through A9 has a 3 as a possible value, then the 3 must belong in A2, and we can update to {'A1': '7', 'A2':'3', ...}. These updates to A2 may in turn cause further updates to its peers, and the peers of those peers, and so on. This process is called constraint propagation.

The function assign(values, s, d) will return the updated values (including the updates from constraint propagation), but if there is a contradiction--if the assignment cannot be made consistently--then assign returns False. For example, if a grid starts with the digits '77...' then when we try to assign the 7 to A2, assign would notice that 7 is not a possibility for A2, because it was eliminated by the peer, A1.

It turns out that the fundamental operation is not assigning a value, but rather eliminating one of the possible values for a square, which we implement with eliminate(values, s, d). Once we have eliminate, then assign(values, s, d) can be defined as "eliminate all the values from s except d".

In [4]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

### Test run

In [5]:
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (parse_grid(grid1))
display(sudoku_values)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



# Constraint Propagation - code refactoring

Try to decompose* eliminate function into smaller one.
* Decomposition is a process by which you can break down one complex function into multiple smaller functions. By doing this, you can solve for functions in shorter, easier-to-understand pieces.

In [6]:
def eliminate2(values):
    """
    Iterate through all squares and every time 
       if there is a square with one value, 
       then eliminate this value from the peers

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for s in squares:
        d = values[s]
        if len(d) == 1:
            for p in peers[s]:
                values[p] = values[p].replace(d, '')
    
    return values


def only_choice(values):
    """
    Iterate through all squares and every time
        if there is a square with a value that only fits in one square, 
        assign the value to this square

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for s in squares:
        value = values[s]
        if len(value) > 1:
            for d in value:
                if not any(p for p in peers[s] if d in values[p]):
                    values[s] = d
                    break
    
    return values


def score(values):
    return sum(len(values[s]) for s in squares)


def reduce_puzzle(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """    
    while score(values) != score(eliminate2(values)) or score(values) != score(only_choice(values)):
        pass
    
    return values
    

def parse_grid2(grid):
    return {s: d if d in digits else digits for s, d in zip(squares, grid)}

### Performance comparison:

In [7]:
from timeit import timeit

expresions = ['parse_grid({grid})', 'reduce_puzzle(parse_grid2({grid}))']
grids = ['grid1', 'grid2']
setup = ('from __main__ import squares, peers, units, '
         'parse_grid, parse_grid2, assign, eliminate, '
         'eliminate2, only_choice, reduce_puzzle, score, grid1, grid2')
iterations = 100

for statement in (expr.format(grid=grid) for grid in grids for expr in expresions):
    et = timeit(statement, setup=setup, number=iterations)
    print(f'{iterations}x {statement:<33} took {et:.3f} s')

100x parse_grid(grid1)                 took 0.402 s
100x reduce_puzzle(parse_grid2(grid1)) took 0.294 s
100x parse_grid(grid2)                 took 0.235 s
100x reduce_puzzle(parse_grid2(grid2)) took 0.066 s


# Naked twins
http://www.sudokudragon.com/tutorialnakedtwins.htm


In [8]:
def find_naked_twins(values):
    twins = set()
    for s in squares:
        if len(values[s]) == 2:
            candidates = set(tuple(s1 for s1 in u if values[s] == values[s1]) for u in units[s] for s1 in u)
            twins = twins | set(c for c in candidates if len(c) > 1)
    return twins


def common_units(a, b):
    return [u for u in units[a] if b in u]


def naked_twins(values):
    """
    eliminate values using the naked twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    for twins in find_naked_twins(values):
        twin, _ = twins
        a, b = values[twin]
        for u in common_units(*twins):
            for s in u:
                if s not in twins:
                    values[s] = values[s].replace(a, '')
                    values[s] = values[s].replace(b, '')
    return values

### More testing data
Naked twins strategy is unnecesary for solving grid1 and uneffective against grid2, so we'll need some more testing data.  
I've borrowed some examples from [aaugustin/sudoku](https://github.com/aaugustin/sudoku/blob/master/benchmark/95_hard_puzzles) and saved them alongside this notebook

In [9]:
with open('95_hard_puzzles', 'r') as f:
    sudokus = [grid[:-1] for grid in f.readlines()]

# Putting our tools to the test

Let's try solving all the new puzzles with constraint propagation only and see how we do.  
The `score` function returns the number of gidits in a whole grid, so we're expecting a score of 81 for a solved sudoku and any greater value for the ones that need some more work.

In [10]:
scores = [score(reduce_puzzle(parse_grid2(grid))) for grid in sudokus]
print(f'{scores.count(81)} sudokus solved using constraint propagation')
print(f'Overall score: {sum(scores)}')

0 sudokus solved using constraint propagation
Overall score: 26820


:(  
Our new approach will be to reduce the puzzle with naked twins strategy every time we get stuck with constraint propagation

In [11]:
new_scores = []
for grid in sudokus:
    values = parse_grid2(grid)
    while any([score(values) != score(reduce_puzzle(values)),
               score(values) != score(naked_twins(values))]):
        pass
    new_scores.append(score(values))
    
print(f'{new_scores.count(81)} sudokus solved using naked twins')
print(f'Overall score: {sum(new_scores)}')

0 sudokus solved using naked twins
Overall score: 26656


Still no success, though the overall score indicates some improvement. Let's have a closer look at the grids that gave better results with the new strategy.

In [12]:
with_naked_twins = [i for i, (a, b) in enumerate(zip(scores, new_scores)) if a != b]
for i in with_naked_twins[:]:
    print(f'Solving grid {i}...\n')
    values = parse_grid2(sudokus[i])
    display(values)
    print(f'Score: {score(values)}\n\n')
    while True:
        if score(values) != score(reduce_puzzle(values)):
            print('Constraint propagation gave:')
            display(values)
            print(f'Score: {score(values)}\n\n')
        elif score(values) != score(naked_twins(values)):
            print('Naked twins gave:')
            display(values)
            print(f'Score: {score(values)}\n\n')
        else:
            break
            
       

Solving grid 11...

123456789     6     123456789 |    5     123456789     1     |123456789     9     123456789 
    1     123456789 123456789 |123456789     9     123456789 |123456789     5         3     
    9     123456789 123456789 |123456789 123456789     7     |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |    8     123456789 123456789 |123456789     7     123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |    5     123456789     8     
123456789     8         1     |    7     123456789     5     |123456789     3     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |123456789     5     123456789 |    2     123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789     7         6     |123456789 123456789

Naked twins gave:
   3       8      145   |   6      2479    1249  |  247     2479    479   
 14567    4567     9    | 12578    2478    1248  | 234678   247    34678  
  467      2       46   |  789      3      489   |   5       1     46789  
------------------------+------------------------+------------------------
1246789   4679   12468  | 23789   26789     5    | 23478    2479   134789 
 245789    3      2458  |  2789     1      289   |  2478     6     45789  
1256789   5679   12568  |   4     26789   23689  |  2378    2579   135789 
------------------------+------------------------+------------------------
   29      1       7    |  239      5      239   |   46      8       46   
  2468     46    23468  |  1238    2468   123468 |   9       57      57   
 45689    4569    4568  |   89     4689     7    |   1       3       2    

Score: 262


Solving grid 34...

123456789 123456789     8     |123456789     9     123456789 |    1     123456789 123456789 
123456789     6     123456789 